### MLib is Machine Learning Library for Spark
1. Incorporates with Numpy in python
2. It provied an Integrated data analysis Flow
3. Enhances speed and performances
4. Clustering, Feature Pattern Matching, Linear Algebra, Collaborative filtering, classification, regression

In [1]:
import pyspark
from pyspark.sql import SparkSession
sc = pyspark.SparkContext(appName="mlib")
spark = SparkSession.builder.appName('mlib').getOrCreate()

In [2]:
df = spark.read.csv("file:///home/aditya/GitHub Projects/Spark-Tutorials/Resources/heart.csv",inferSchema=True,header=True)
df.show(5)

+----+---+---+--------+-----+---+-------+-------+-----+-------+-----+---+----+---+
| age|sex| cp|trestbps| chol|fbs|restecg|thalach|exang|oldpeak|slope| ca|thal|num|
+----+---+---+--------+-----+---+-------+-------+-----+-------+-----+---+----+---+
|67.0|1.0|4.0|   160.0|286.0|0.0|    2.0|  108.0|  1.0|    1.5|  2.0|3.0| 3.0|  2|
|67.0|1.0|4.0|   120.0|229.0|0.0|    2.0|  129.0|  1.0|    2.6|  2.0|2.0| 7.0|  1|
|37.0|1.0|3.0|   130.0|250.0|0.0|    0.0|  187.0|  0.0|    3.5|  3.0|0.0| 3.0|  0|
|41.0|0.0|2.0|   130.0|204.0|0.0|    2.0|  172.0|  0.0|    1.4|  1.0|0.0| 3.0|  0|
|56.0|1.0|2.0|   120.0|236.0|0.0|    0.0|  178.0|  0.0|    0.8|  1.0|0.0| 3.0|  0|
+----+---+---+--------+-----+---+-------+-------+-----+-------+-----+---+----+---+
only showing top 5 rows



In [3]:
import numpy as np
df_pd = df.toPandas()
df_pd.replace('?',np.NaN,inplace=True)
df_pd.dropna(inplace=True)
df_pd.to_csv("Resources/heart_new.csv",header=False,index=False)

In [4]:
from pyspark.mllib.regression import LabeledPoint
points = sc.textFile("file:///home/aditya/GitHub Projects/Spark-Tutorials/Resources/heart_new.csv")
def prepareLabelledPoints(row):
    values = [float(s) for s in row.strip().split(',')]
    return LabeledPoint(int(values[13]),values[:13])
pointrdd = points.map(prepareLabelledPoints)

In [5]:
pointrdd.take(5)

[LabeledPoint(2.0, [67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0]),
 LabeledPoint(1.0, [67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0]),
 LabeledPoint(0.0, [37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0]),
 LabeledPoint(0.0, [41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0]),
 LabeledPoint(0.0, [56.0,1.0,2.0,120.0,236.0,0.0,0.0,178.0,0.0,0.8,1.0,0.0,3.0])]

In [31]:
from pyspark.mllib.tree import DecisionTree
(trainData,testData) = pointrdd.randomSplit([0.7,0.3])

In [36]:
trained_model = DecisionTree.trainClassifier(trainData,numClasses=5,categoricalFeaturesInfo={},impurity='gini',maxDepth=10)

In [37]:
predictions = trained_model.predict(testData.map(lambda x: x.features))
predictions.take(10)

[0.0, 2.0, 2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 2.0, 3.0]

In [38]:
labelsandpredictions = testData.map(lambda lp: lp.label).zip(predictions)
labelPrediction=labelsandpredictions.collect()

In [39]:
total = len(labelPrediction)
correct = len(list(filter(lambda x: x[0]==x[1],labelPrediction)))
print("Accuracy :{}".format(correct/total))

Accuracy :0.5555555555555556
